<a href="https://colab.research.google.com/github/yassir-a-p/blender-on-colab/blob/main/blender-on-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check GPU device

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv 

#Setup

Specify your desired blender version and the path to your .blend file within Google Drive or Colab local storage.

In [ ]:
#@title Setup
#@markdown Please configure your setup

blender_version = 'blender2.92.0' #@param ["blender2.79b", "blender2.80", "blender2.81a", "blender2.82a", "blender2.83.13", "blender2.90.1", "blender2.91.2", "blender2.92.0"] {allow-input: false}
#@markdown You can right click a file and click copy path
path_to_blend = '/content/drive/your-folder/your-blend-file.blend'  #@param {type: "string"}
output_path = '/content/drive/your-folder/output/image_###'  #@param {type: "string"}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = True #@param {type:"boolean"}
use_drive = True #@param {type:"boolean"}
#@markdown ---

## Google Drive
Run this code if you want to connect to your Google Drive (check **use_drive** option above)<br><br>
There will be a link and an empty field.<br>Click the link, choose the account you want to access its GDrive, then click **Allow**. Finally, copy the code and paste in the field, then press Enter.

In [ ]:
if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')


# **Installing Blender in the cloud**

Specifying download location for chosen blender version

In [ ]:
if blender_version == "blender2.79b":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2
elif blender_version == "blender2.80":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.80/blender-2.80-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.81a":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.81/blender-2.81a-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.82":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.82/blender-2.82a-linux64.tar.xz"
elif blender_version == "blender2.83.13":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.83/blender-2.83.13-linux64.tar.xz"
elif blender_version == "blender2.90.1":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.90/blender-2.90.1-linux64.tar.xz"
elif blender_version == "blender2.91.2":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.91/blender-2.91.2-linux64.tar.xz"
elif blender_version == "blender2.92.0":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.92/blender-2.92.0-linux64.tar.xz"

Download, unpack, and move blender to designated location

In [ ]:
!mkdir $blender_version
if blender_version == "blender2.79b" or "blender2.80" or "blender2.81a":
    !wget -O '{blender_version}.tar.bz2' -nc $download_path
    !tar -xf '{blender_version}.tar.bz2' -C ./$blender_version --strip-components=1
else:
    !wget -O '{blender_version}.tar.xz' -nc $download_path
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1


*This block is required as some weird behaviors with libtcmalloc appeared in the colab VM*

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt update
!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

!echo $LD_PRELOAD

GPU Dependencies

In [ ]:
!apt install libboost-all-dev
!apt install libgl1-mesa-dev
!apt install libglu1-mesa libsm-dev

# **Render**

In [ ]:
#@title **Render Setup**

#@markdown **renderOneFrame** : if you only want to render a single frame. Specify the frame on startFrame field below<br>
#@markdown **renderAllFrame** : if you want to render all the frames as already specified in your .blend file.<br>
#@markdown **renderWithinRange** : if you want to render specific frames within certain range], set the _**startFrame**_ dan _**endFrame**_ below

whatToRender = 'renderOneFrame' #@param ["renderOneFrame", "renderAllFrame", "renderWithinRange"] {allow-input: false}

startFrame =  101 #@param {type: "integer"}
endFrame =  250 #@param {type: "integer"}
#@markdown ---

override_certain_settings = False #@param {type:"boolean"}
#@markdown Override the settings of samples, resolution, dan compression 
samples = 500 #@param {type:"integer"}
resolution_x = 1280 #@param {type:"integer"}
resolution_y = 720 #@param {type:"integer"}
compression_level = 50 #@param ["0", "25", "50", "75", "100"] {allow-input: false}

Required for Blender to use the GPU as expected

In [ ]:
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "# Attempt to set GPU device types if available\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE',):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "#for scene in bpy.data.scenes:\n"+\
    "#    scene.render.tile_x = 64\n"+\
    "#    scene.render.tile_y = 64\n"+\
    "# Enable all CPU and GPU devices\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

override = "# My attempt to override Samples, Resolution, dan Compression\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.samples = "+int(samples)+"\n"+\
    "scene.render.resolution_x = "+int(resolution_x)+"\n"+\
    "scene.render.resolution_y = "+int(resolution_y)+"\n"+\
    "scene.render.image_settings.compression = "+int(compression_level)+"\n"
with open(`override_others.py`), `w`) as g:
    g.write(override) 

## **Render CYCLES with CUDA**

In [ ]:
if override_certain_settings == False:
  if whatToRender == 'renderOneFrame':
    !sudo ./$blender_version/blender -P './setgpu.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -f {startFrame}
  elif whatToRender == 'renderAllFrame':
    !sudo ./$blender_version/blender -P './setgpu.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -a
  elif whatToRender == 'renderWithinRange':
    !sudo ./$blender_version/blender -P './setgpu.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -s {startFrame} -e {endFrame} -a
else:
  if whatToRender == 'renderOneFrame':
    !sudo ./$blender_version/blender -P './setgpu.py' -P './override_others.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -f {startFrame}
  elif whatToRender == 'renderAllFrame':
    !sudo ./$blender_version/blender -P './setgpu.py' -P './override_others.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -a
  elif whatToRender == 'renderWithinRange':
    !sudo ./$blender_version/blender -P './setgpu.py' -P './override_others.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -s {startFrame} -e {endFrame} -a

## **Render Cycles with OPTIX**

not using setgpu.py

In [ ]:
if override_certain_settings == False:
  if whatToRender == 'renderOneFrame':
    !sudo ./$blender_version/blender -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -f {startFrame} -- --cycles-device OPTIX 
  elif whatToRender == 'renderAllFrame':
    !sudo ./$blender_version/blender -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -a -- --cycles-device OPTIX 
  elif whatToRender == 'renderWithinRange':
    !sudo ./$blender_version/blender -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -s {startFrame} -e {endFrame} -a -- --cycles-device OPTIX 
else:
  if whatToRender == 'renderOneFrame':
    !sudo ./$blender_version/blender -P './override_others.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -f {startFrame} -- --cycles-device OPTIX 
  elif whatToRender == 'renderAllFrame':
    !sudo ./$blender_version/blender -P './override_others.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -a -- --cycles-device OPTIX 
  elif whatToRender == 'renderWithinRange':
    !sudo ./$blender_version/blender -P './override_others.py' -b '{path_to_blend}' -noaudio -E CYCLES -o '{output_path}' -F 'PNG' -s {startFrame} -e {endFrame} -a -- --cycles-device OPTIX 

## **COPYRIGHT NOTICE**

    Copyright 2021 Yassir A. P.

    Copyright 2021 github.com/donmahallem

    Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.

    You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

    This file has been modified to improve workflow, add support for OPTIX rendering, override settings of Samples, Resolution, and Compression.
